In [1]:
import os
from langchain_groq import ChatGroq
from typing import List, Optional
import json
import unicodedata
import os
import json
import asyncio
import re
import unicodedata
from pathlib import Path
from typing import List, Optional

import uvicorn
import numpy as np
import firebase_admin
from dotenv import load_dotenv
from pydantic import BaseModel

from fastapi import FastAPI, Form, HTTPException, Depends, Header, BackgroundTasks
from fastapi.responses import FileResponse, JSONResponse
from starlette.middleware.cors import CORSMiddleware
from starlette.staticfiles import StaticFiles

from firebase_admin import credentials, firestore, auth

from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http import models
from langchain.docstore.document import Document
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from llama_parse import LlamaParse

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# https://github.com/huggingface/transformers/issues/5486:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

C:\Users\magang\miniconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pathlib import Path
from tempfile import mkdtemp

from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_docling.loader import ExportType


def _get_env_from_colab_or_os(key):
    try:
        from google.colab import userdata

        try:
            return userdata.get(key)
        except userdata.SecretNotFoundError:
            pass
    except ImportError:
        pass
    return os.getenv(key)


load_dotenv()

HF_TOKEN = _get_env_from_colab_or_os("HF_TOKEN")
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-large-en-v1.5")
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
data_dir = Path("./data")
FILE_PATH = data_dir / "thesis.pdf"
EMBED_MODEL_ID = "ms-marco-MiniLM-L-12-v2"
GEN_MODEL_ID = "mistralai/Mixtral-8x7B-Instruct-v0.1"
EXPORT_TYPE = ExportType.MARKDOWN
QUESTION = "Which are the main AI models in Docling?"
PROMPT = PromptTemplate.from_template(
    "Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {input}\nAnswer:\n",
)
TOP_K = 3
MILVUS_URI = str(Path(mkdtemp()) / "docling.db")

In [3]:
from langchain_docling import DoclingLoader

loader = DoclingLoader(
    file_path=FILE_PATH,
    export_type=EXPORT_TYPE,
)

docs = loader.load()

2025-09-24 10:50:08,540 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-09-24 10:50:08,776 - INFO - Going to convert document batch...
2025-09-24 10:50:08,776 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-09-24 10:50:08,855 - INFO - Loading plugin 'docling_defaults'
2025-09-24 10:50:08,855 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-09-24 10:50:08,855 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-09-24 10:50:08,965 - INFO - Loading plugin 'docling_defaults'
2025-09-24 10:50:08,981 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-09-24 10:50:08,981 - INFO - Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-09-24 10:50:09,230 - INFO - Accelerator device: 'cpu'
2025-09-24 10:50:11,612 - I

In [4]:
print(docs)

[Document(metadata={'source': WindowsPath('data/thesis.pdf')}, page_content='## PENYELENGGARAAN PENDIDIKAN\n\n## 2.1 Sistem Penyelenggaraan Pendidikan\n\n## 2.1.1 Sistem Semester\n\n1. Sistem semester adalah penyelenggaraan program pendidikan dengan menggunakan satuan waktu terkecil yang disebut semester.\n2. Semester merupakan satu-kesatuan waktu kegiatan yang terdiri atas 16-18 minggu kuliah  atau  kegiatan  terjadwal  lainnya,  berikut  kegiatan  iringannya  termasuk  2-3 minggu kegiatan penilaian.\n\n## 2.2 Program Semester\n\n1. Program semester adalah program penyelenggaraan pendidikan secara bulat untuk setiap mata kuliah pada semester tersebut.\n2. Penyelenggaraan pendidikan dalam satu semester terdiri atas kegiatan perkuliahan teori,  praktikum,  dan  kerja  lapangan,  baik  dalam  bentuk  tatap  muka,  terstruktur, maupun kerja mandiri yang diakhiri dengan evaluasi.\n3. Dalam satu semester ditawarkan sejumlah mata kuliah dengan bobot satuan kredit semester (sks) yang bervaria

In [5]:
parsed_md_path = data_dir / "parsed_document.md"

from pathlib import Path


with parsed_md_path.open("w", encoding="utf-8") as f:
    for i, doc in enumerate(docs, start=1):
        source = doc.metadata.get("source", f"Document {i}")
        f.write(f"# Source: {source}\n\n")   # judul per dokumen
        f.write(doc.page_content.strip() + "\n\n")
        f.write("---\n\n")  # pemisah antar dokumen

print(f"Dokumen berhasil diexport ke {parsed_md_path}")


Dokumen berhasil diexport ke data\parsed_document.md


In [6]:
print(docs[0].page_content[:500])  # akses isi teks

## PENYELENGGARAAN PENDIDIKAN

## 2.1 Sistem Penyelenggaraan Pendidikan

## 2.1.1 Sistem Semester

1. Sistem semester adalah penyelenggaraan program pendidikan dengan menggunakan satuan waktu terkecil yang disebut semester.
2. Semester merupakan satu-kesatuan waktu kegiatan yang terdiri atas 16-18 minggu kuliah  atau  kegiatan  terjadwal  lainnya,  berikut  kegiatan  iringannya  termasuk  2-3 minggu kegiatan penilaian.

## 2.2 Program Semester

1. Program semester adalah program penyelenggaraan 


In [7]:
if EXPORT_TYPE == ExportType.DOC_CHUNKS:
    splits = docs
elif EXPORT_TYPE == ExportType.MARKDOWN:
    from langchain_text_splitters import MarkdownHeaderTextSplitter

    splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("#", "Header_1"),
            ("##", "Header_2"),
            ("###", "Header_3"),
        ],
    )
    splits = [split for doc in docs for split in splitter.split_text(doc.page_content)]
else:
    raise ValueError(f"Unexpected export type: {EXPORT_TYPE}")

In [8]:
print(splits)

[Document(metadata={'Header_2': '2.1.1 Sistem Semester'}, page_content='1. Sistem semester adalah penyelenggaraan program pendidikan dengan menggunakan satuan waktu terkecil yang disebut semester.\n2. Semester merupakan satu-kesatuan waktu kegiatan yang terdiri atas 16-18 minggu kuliah  atau  kegiatan  terjadwal  lainnya,  berikut  kegiatan  iringannya  termasuk  2-3 minggu kegiatan penilaian.'), Document(metadata={'Header_2': '2.2 Program Semester'}, page_content='1. Program semester adalah program penyelenggaraan pendidikan secara bulat untuk setiap mata kuliah pada semester tersebut.\n2. Penyelenggaraan pendidikan dalam satu semester terdiri atas kegiatan perkuliahan teori,  praktikum,  dan  kerja  lapangan,  baik  dalam  bentuk  tatap  muka,  terstruktur, maupun kerja mandiri yang diakhiri dengan evaluasi.\n3. Dalam satu semester ditawarkan sejumlah mata kuliah dengan bobot satuan kredit semester (sks) yang bervariasi, sesuai dengan yang telah ditetapkan dalam kurikulum yang berlak

In [9]:
NER_llm = ChatGroq(temperature=0, model_name="llama-3.3-70b-versatile", api_key=GROQ_API_KEY)
async def extract_ner_with_groq(text: str) -> List[str]:
    """
    Mengekstrak dan menormalisasi entitas/kata kunci dari teks,
    melakukan perluasan singkatan, dan menambahkan varian typo yang umum
    menggunakan Llama 3 via Groq.
    """
    if not NER_llm:
        return []
    prompt = f"""
    You are an expert AI assistant specializing in keyword extraction, abbreviation expansion, and common typo handling for Indonesian academic and general contexts.
    Your task is to analyze the user's text and perform the following steps:
    1.  **Identify and Extract**: Extract all relevant keywords, topics, and common named entities.
    2.  **Expand Abbreviations**: For each identified abbreviation or acronym, provide its full, expanded form. For example, "PA" should be expanded to "Pembimbing Akademik" and "FK" to "Fakultas Kedokteran".
    3.  **Handle Typos**: For each extracted keyword, generate a common Indonesian typographical or spelling variant. Do not replace the original keyword; instead, add the variant to the list. For example, "kurikulum" might have a common typo like "kurikulim". The output should contain both.
    4.  **Final Output Format**: Combine all keywords, expanded forms, and typo variants into a single, flat JSON array of strings. Do not include any duplicates.
    Adhere strictly to the following rules:
    - The output MUST be a single, valid JSON array of strings.
    - DO NOT include any explanatory text, narrative, or markdown formatting (like ```json) before or after the JSON array. Your entire response must start with '[' and end with ']'.
    ---
    EXAMPLE
    Text: "Tolong jelaskan tentang kurikulim di fk."
    JSON Output: ["kurikulim", "kurikulum", "fk", "fakultas kedokteran"]
    ---
    Now, analyze the following text.
    Text to analyze: "{text}"
    JSON Output:
    """
    try:
        response = await NER_llm.ainvoke(prompt)
        cleaned_response = response.content.strip().replace("```json", "").replace("```", "")
        extracted_list = json.loads(cleaned_response)
        
        if isinstance(extracted_list, list):
            normalized_keywords = []
            seen_keywords = set()
            for item in extracted_list:
                normalized_item = unicodedata.normalize('NFKC', item)
                normalized_item = re.sub(r'[^a-zA-Z0-9\s-]', '', normalized_item)
                normalized_item = ' '.join(normalized_item.split()).strip().lower()
                
                if normalized_item and normalized_item not in seen_keywords:
                    normalized_keywords.append(normalized_item)
                    seen_keywords.add(normalized_item)
            return normalized_keywords
        return []
    except Exception as e:
        print(f"Gagal mengekstrak NER: {e}")
        return []

In [10]:
doc = splits[0]
print(doc.page_content)
print(doc.metadata)

1. Sistem semester adalah penyelenggaraan program pendidikan dengan menggunakan satuan waktu terkecil yang disebut semester.
2. Semester merupakan satu-kesatuan waktu kegiatan yang terdiri atas 16-18 minggu kuliah  atau  kegiatan  terjadwal  lainnya,  berikut  kegiatan  iringannya  termasuk  2-3 minggu kegiatan penilaian.
{'Header_2': '2.1.1 Sistem Semester'}


In [ ]:
# Looping melalui setiap dokumen
for i, doc in enumerate(splits):
    if doc.page_content.strip():
        print(f"   -> Mengekstrak keyword untuk chunk {i+1}/{len(splits)}...")

        # Ekstraksi keyword dari isi konten
        content_keywords = await extract_ner_with_groq(doc.page_content)
        doc.metadata['content_keywords'] = content_keywords

        # Ekstraksi keyword dari judul (header) dari SEMUA dokumen
        title_keywords = []
        for other_doc in splits:
            # Asumsikan metadata judul disimpan dengan kunci seperti 'h1', 'h2', dst.
            for key, value in other_doc.metadata.items():
                if key.startswith('H') and isinstance(value, str):
                    extracted = await extract_ner_with_groq(value)
                    title_keywords.extend(extracted)

        doc.metadata['title_keywords'] = title_keywords

        # Cetak hasil
        print(f"    \033[92mKeywords Konten: {content_keywords}\033[0m")
        print(f"    \033[94mKeywords Judul: {title_keywords}\033[0m")

print("\nEkstraksi keyword selesai. Membangun database...")

In [ ]:
# Inisialisasi Qdrant Vectorstore
qdrant = Qdrant.from_documents(
    splits,
    embeddings,
    path="./db",
    collection_name="document_embeddings",
    force_recreate=False
)

# Buat retriever dasar dari Qdrant
base_retriever = qdrant.as_retriever(search_kwargs={"k": 10})

# Bungkus dengan Contextual Compression Retriever
retriever = ContextualCompressionRetriever(
    base_compressor=compressor,.2
    base_retriever=base_retriever
)

print("\n\033[92mInisialisasi RAG Retriever berhasil dan siap digunakan.\033[0m")



Inisialisasi RAG Retriever berhasil dan siap digunakan.


In [10]:
print(splits[0])
print(splits[0].metadata)


page_content='1. Sistem semester adalah penyelenggaraan program pendidikan dengan menggunakan satuan waktu terkecil yang disebut semester.
2. Semester merupakan satu-kesatuan waktu kegiatan yang terdiri atas 16-18 minggu kuliah  atau  kegiatan  terjadwal  lainnya,  berikut  kegiatan  iringannya  termasuk  2-3 minggu kegiatan penilaian.' metadata={'Header_2': '2.1.1 Sistem Semester', 'content_keywords': ['sistem semester', 'program pendidikan', 'satuan waktu', 'semester', 'waktu kegiatan', 'minggu kuliah', 'kegiatan terjadwal', 'kegiatan iringan', 'kegiatan penilaian', 'sistem', 'penyelenggaraan', 'pendidikan', 'satuan', 'waktu terkecil', 'kesatuan waktu', 'kuliah', 'penilaian', 'sistim semester', 'semestre', 'semster', 'penyelengaraan', 'waktu', 'kesatuan'], 'title_keywords': []}
{'Header_2': '2.1.1 Sistem Semester', 'content_keywords': ['sistem semester', 'program pendidikan', 'satuan waktu', 'semester', 'waktu kegiatan', 'minggu kuliah', 'kegiatan terjadwal', 'kegiatan iringan', 'keg

In [11]:
question="apa matakuliah yang harus diambil di semester 1 prodi bimbingan konseling?"

In [12]:
llm = ChatGroq(temperature=0.5, model_name="meta-llama/llama-4-scout-17b-16e-instruct", api_key=GROQ_API_KEY)

In [ ]:
answer = await llm.ainvoke()